In [26]:
conda install pyarrow


Note: you may need to restart the kernel to use updated packages.


In [23]:
conda install fastparquet

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\FelipeMartin\anaconda3

  added / updated specs:
    - fastparquet


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    blosc-1.21.3               |       h6c2663c_0          86 KB
    cramjam-2.6.2              |   py39h1718810_0         1.2 MB
    fastparquet-2023.4.0       |   py39h9128911_0         420 KB
    imagecodecs-2021.8.26      |   py39h319e4f4_2         6.1 MB
    jxrlib-1.1                 |       he774522_2         337 KB
    m2w64-gcc-libgfortran-5.3.0|                6         340 KB
    m2w64-gcc-libs-5.3.0       |                7         518 KB
    m2w64-gcc-libs-core-5.3.0  |                7         213 KB
    m2w64-gmp-6.1.0            |                2         689 KB
    prompt_toolkit-3.0.36      |       hd3eb1b0_0           5 KB
    ---------------------------------------


The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/win-64::blosc==1.21.0=h19a0ad4_0
  - defaults/win-64::imagecodecs==2021.8.26=py39ha1f97ea_0
  - defaults/win-64::jupyter==1.0.0=py39haa95532_7
  - defaults/noarch::jupyter_console==6.4.0=pyhd3eb1b0_0
  - defaults/noarch::prompt_toolkit==3.0.20=hd3eb1b0_0
  - defaults/win-64::pytables==3.6.1=py39h56d22b6_1
  - defaults/win-64::scikit-image==0.19.2=py39hf11a4ad_0
  - defaults/noarch::tifffile==2021.7.2=pyhd3eb1b0_2


==> WARNING: A newer version of conda exists. <==
  current version: 22.9.0
  latest version: 23.7.2

Please update conda by running

    $ conda update -n base -c defaults conda




# LOAD DATA

In [2]:
import os
import glob
import numpy as np
import pandas as pd

# Root folder containing the folders with the .npy files'.
carpeta_raiz = 'C:/Users/FelipeMartin/OneDrive - University of Sussex/DISSERTATION/segundo intento/


patron = '**/*.npy'

# Get list of .npy files matching the pattern
archivos = glob.glob(os.path.join(carpeta_raiz, patron), recursive=True)


def transform_text(input_text):
    parts = input_text.split("-")
    transformed_text = f"{parts[0]}/{parts[1]}-{parts[2]}-{parts[3]}/{parts[4]}"
    return transformed_text


folders=[]
for archivo in archivos:
    nombre = os.path.splitext(os.path.basename(archivo))[0]  # Obtener el nombre del archivo sin la extensión
    # Get the name of the last 3 folders
    carpetas = os.path.dirname(archivo).split(os.sep)[-3:]
    folder_name = '-'.join(carpetas)
    folders.append(folder_name)

folders=set(folders)
folders=list(folders)
print(folders)

for folder in folders:
    
    folder_transform = transform_text(folder)
    root_folder=carpeta_raiz +folder_transform
    # Pattern to search for .npy files in all subfolders
    patron = '**/*.npy'
    # Get list of .npy files matching the pattern
    archivos = glob.glob(os.path.join(root_folder, patron), recursive=True)

    # Dictionary to store the accumulated DataFrames per filename
    dataframes = {}

    # Iterate over the files and accumulate the data in the dictionary
    for archivo in archivos:
        nombre = os.path.splitext(os.path.basename(archivo))[0]  # Obtener el nombre del archivo sin la extensión

        carpetas = os.path.dirname(archivo).split(os.sep)[-3:]
        folder_name = '-'.join(carpetas)
        
        # Load the NPY file
        data = np.load(archivo)

         # Flatten the data to a two-dimensional matrix
        data_flattened = data.reshape(data.shape[0], -1)
        data_flattened = pd.DataFrame(data_flattened)
        data_flattened['folder_name'] = folder_name

        if nombre == '_ss_2pCalcium.dff':
            # print('Realizando transformación...')
            data_flattened = data_flattened.drop('folder_name', axis=1)

            # Use the melt function to convert columns into rows.
            data_flattened = pd.melt(data_flattened, var_name='nROIs', value_name='dff')
            # Add a counter from 0 to 42874 for each ID
            data_flattened['nROW'] = data_flattened.groupby('nROIs').cumcount()
            # Add column with the name of the last 3 folders# Add a counter from 0 to 42874 for each ID
            data_flattened['folder_name'] = folder_name

        if nombre == '_ss_2pCalcium.timestamps':
            data_flattened = data_flattened.rename(columns={0: 'timestamps'})
            data_flattened['folder_name'] = folder_name

        if nombre == '_ss_grating.intervals':
            data_flattened = data_flattened.rename(columns={0: 'start', 1: 'end'})
            data_flattened['folder_name'] = folder_name

        if nombre == '_ss_grating._ss_gratingID':
            data_flattened = data_flattened.rename(columns={0: 'gratingID'})
            data_flattened['folder_name'] = folder_name

        if nombre in dataframes:

            # If the filename already exists in the dictionary, accumulates the data
            dataframes[nombre] = pd.concat([dataframes[nombre], data_flattened], ignore_index=True)

        else:
            dataframes[nombre] = data_flattened

    #dataframes['_ss_2pCalcium.dff']
    Calciumtimestamps=dataframes['_ss_2pCalcium.timestamps']
    #dataframes['_ss_2pRois._ss_2pPlanes']
    #dataframes['_ss_2pRois.ids']
    Rois_xyz=dataframes['_ss_2pRois.xyz']
    Rois_xyz=Rois_xyz.rename(columns={0: 'axis_x', 1: 'axis_y', 2: 'axis_z'})
    #dataframes['_ss_2pRois.isGad']
    #dataframes['_ss_2pPlanes.delay']
    runningspeed = dataframes['_ss_running.speed']
    runningspeed=runningspeed.rename(columns={0: 'speed'})
    runningtimestamps = dataframes['_ss_running.timestamps']
    runningtimestamps=runningtimestamps.rename(columns={0: 'timestamps'})
    Eyediameter=dataframes['eye.diameter']
    Eyediameter=Eyediameter.rename(columns={0: 'diameter'})
    #dataframes['eye.xyPos']
    Eyetimestamps = dataframes['eye.timestamps']
    Eyetimestamps=Eyetimestamps.rename(columns={0: 'timestamps'})
    #dataframes['_ss_grating.intervals']
    #dataframes['_ss_grating._ss_gratingID']
    #dataframes['_ss_gratingID.directions']
    #dataframes['_ss_sparseNoise.times']
    #dataframes['_ss_sparseNoise._ss_sparseNoiseID']
    #dataframes['_ss_sparseNoiseArea.edges']
    #dataframes['_ss_sparseNoiseID.map']
    #dataframes['_ss_recordings.gratings_intervals']
    #dataframes['_ss_recordings.sparseNoise_intervals']
    #dataframes['_ss_gratingTrials.amplitudes']
    #dataframes['_ss_gratingTrials.largePupil']

   #We add the grid angle ID to the interval table.

    dataframes['_ss_grating.intervals'] = pd.merge(dataframes['_ss_grating.intervals'], dataframes['_ss_grating._ss_gratingID'], left_index=True, right_index=True)
    #dataframes['_ss_grating.intervals']

    #Add an ID column for each interval
    #List of unique values in the column "folder_name".
    carpeta_values = dataframes['_ss_2pCalcium.timestamps']['folder_name'].unique()

    # Results for each value of "folder_name".
    resultados = []

    for carpeta in carpeta_values:
        intervalos = dataframes['_ss_grating.intervals']
    filtro = intervalos['folder_name_x'] == carpeta
    intervalos.loc[filtro, 'interval_ID'] = range(1, filtro.sum() + 1)




    #---------------------------------------------------------------------------------------------------------------------------------
    # We add the time interval of the grids to the time table.

    # List of unique values in the column "folder_name".
    carpeta_values = dataframes['_ss_2pCalcium.timestamps']['folder_name'].unique()

     # Results for each value of "folder_name".
    resultados = []

    for carpeta in carpeta_values:
        print(carpeta)
        # Filter the DataFrame by the current value of "folder_name".
        filtered_data_time = dataframes['_ss_2pCalcium.timestamps'][dataframes['_ss_2pCalcium.timestamps']['folder_name'] == carpeta].copy()
        filtered_data_interval = dataframes['_ss_grating.intervals'][dataframes['_ss_grating.intervals']['folder_name_x'] == carpeta].copy()

        # Apply existing code to the filtered DataFrame
        filtered_data_time['in_interval'] = filtered_data_time['timestamps'].apply(
            lambda x: any(row['start'] <= x <= row['end'] for _, row in filtered_data_interval.iterrows())
        )

        filtered_data_time['start'] = filtered_data_time['timestamps'].apply(
            lambda x: next((row['start'] for _, row in filtered_data_interval.iterrows() if row['start'] <= x <= row['end']), None)
        )

        filtered_data_time['end'] = filtered_data_time['timestamps'].apply(
            lambda x: next((row['end'] for _, row in filtered_data_interval.iterrows() if row['start'] <= x <= row['end']), None)
        )

        filtered_data_time['gratingID'] = filtered_data_time['timestamps'].apply(
            lambda x: next((row['gratingID'] for _, row in filtered_data_interval.iterrows() if row['start'] <= x <= row['end']), None)
        )

        filtered_data_time['interval_ID'] = filtered_data_time['timestamps'].apply(
            lambda x: next((row['interval_ID'] for _, row in filtered_data_interval.iterrows() if row['start'] <= x <= row['end']), None)
        )

        # Add the resulting DataFrame to the result list
        resultados.append(filtered_data_time)

    # Concatenate all results into a single DataFrame
    resultados_df = pd.concat(resultados)
    print(resultados_df)
    #---------------------------------------------------------------------------------------------------------------------------------
    #---------------------------------------------------------------------------------------------------------------------------------

        #---------------------------------------------------------------------------------------------------------------------------------
    #---------------------------------------------------------------------------------------------------------------------------------
   ##Add pupil size
    # Create a new column that will have the values reassigned after the filter is applied
    resultados_df['n_interval'] = resultados_df[resultados_df['in_interval'] == True].groupby('gratingID')['interval_ID'].rank(method='dense').astype(int)
    largePupil=dataframes['_ss_gratingTrials.largePupil']
    #    Reassign index and columns
    largePupil.index = largePupil.index + 1
    largePupil=largePupil.drop(['folder_name'], axis=1)
    # Function to rename columns by adding +1
    def rename_columns_with_offset(columns):
        return [col + 1 for col in columns]
    # Apply the rename function to columns
    largePupil.columns = rename_columns_with_offset(largePupil.columns)
    # Reset index to be a column instead of an index
    largePupil.reset_index(inplace=True)
    # TTransforming numeric columns into rows using melt
    melted_largePupil = pd.melt(largePupil, id_vars=['index'], var_name='n_interval', value_name='largePupil')
    # Rename columns
    melted_largePupil.rename(columns={'index': 'gratingID'}, inplace=True)
    # Perform an inner join between the DataFrames in the columns 'gratingID' and 'n_interval'.
    resultados_df = resultados_df.merge(melted_largePupil, on=['gratingID', 'n_interval'],  how='outer')
     #result_df = results_df.merge(melted_largePupil, on=['gratingID', 'n_interval'], how='outer')
    # sort the resulting DataFrame according to the original order of results_df
    resultados_df = resultados_df.sort_values(by=[ 'timestamps'])
    #---------------------------------------------------------------------------------------------------------------------------------
    #---------------------------------------------------------------------------------------------------------------------------------

    #Add NROW to the table
    resultados_df['nROW'] = resultados_df.groupby('folder_name').cumcount()

    #We add the grating ID to the interval table.
    #Join the general table with the time table through the nROW of the general table and the index of the time table.
    resultados_df['folder_nROW'] = resultados_df['folder_name'] + '-' + resultados_df['nROW'].astype(str)

    dataframes['_ss_2pCalcium.dff']['folder_nROW'] = dataframes['_ss_2pCalcium.dff']['folder_name'] + '-' + dataframes['_ss_2pCalcium.dff']['nROW'].astype(str)

    # Enumerate rows for each interval_ID
    resultados_df['interval_ID_enum'] = resultados_df.groupby(['interval_ID']).cumcount() + 1

    unique_counts = resultados_df['interval_ID_enum'].nunique()
    print("unique_count")
    print(unique_counts)
    
    # We get +-2 seconds in each interval
    # Fill downwards (forward) only 31 values
    resultados_df['interval_ID_2'] = resultados_df['interval_ID'].fillna(method='ffill', limit=unique_counts)
    # Fill upwards (backwards) only 31 values
    resultados_df['interval_ID_3'] = resultados_df['interval_ID_2'].fillna(method='bfill', limit=unique_counts)
    # Fill downwards (forward) only 31 values
    resultados_df['gratingID_2'] = resultados_df['gratingID'].fillna(method='ffill', limit=unique_counts)
    ## Fill upwards (backwards) only 3 values
    resultados_df['gratingID_3'] = resultados_df['gratingID_2'].fillna(method='bfill', limit=unique_counts)
    # enumerate the rows for each interval_ID
    resultados_df['interval_ID_enum_2'] = resultados_df.groupby(['interval_ID_3']).cumcount() + 1

    df1=dataframes['_ss_2pCalcium.dff']
    #df2=resultados_df.dropna()
    df2=resultados_df

    intervals=resultados_df[resultados_df['in_interval']==True ][['interval_ID','start','end']]
    intervals_unique=intervals.drop_duplicates(['interval_ID','start','end'])
    intervals_unique
    
    #Join the dff table with the time/interval/grating table.
    # Make the crossover into smaller parts
    chunk_size = 1000
    merged_dfs = []
    for i in range(0, len(df1), chunk_size):
        df1_chunk =df1[i:i+chunk_size]
        merged_df_chunk = df1_chunk.merge(df2, on='folder_nROW', how='inner')
        merged_dfs.append(merged_df_chunk)

    merged_df = pd.concat(merged_dfs)

    #Eliminating unnecessary columns
    merged_df.drop(columns=['nROW_y'], inplace=True)
    merged_df.drop(columns=['folder_name_y'], inplace=True)
    # Reset index
    merged_df.reset_index(drop=True, inplace=True)

    #Eliminating unnecessary columns
    merged_df.drop(columns=['interval_ID_2'], inplace=True)
    merged_df.drop(columns=['gratingID_2'], inplace=True)  
    
    # Create the mapping dictionary
    mapping = {1: '0°', 2: '30°', 3: '60°', 4: '90°', 5: '120°', 6: '150°', 7: '180°',
               8: '210°', 9: '240°', 10: '270°', 11: '300°', 12: '330°', 13: 'NaN'}

    # Map the values in merged_df
    merged_df['gratingValue'] = merged_df['gratingID'].map(mapping)

    #Interpolation of pupil size
    Eyetimestamps_pupilsize = pd.merge(Eyetimestamps, Eyediameter, left_index=True, right_index=True)
    Eyetimestamps_pupilsize.set_index('timestamps', inplace=True)
    import pandas as pd
    from scipy import interpolate

    f = interpolate.interp1d(Eyetimestamps_pupilsize.index, Eyetimestamps_pupilsize['diameter'], kind='linear', fill_value="extrapolate")  # Ajusta 'tamaño_pupila' al nombre de la columna correspondiente en DF2
    merged_df['tamaño_pupila'] = f(merged_df['timestamps'])

    # Function to identify whether the value is numeric or blank
    def identificar_valor(valor):
        if  pd.isnull(valor) :   # Checks if the value is numeric
            return 'null'
        else:
            return 'numeric'

    # Apply the function to the column and create a new column called 'value_type'.
    merged_df['value_type'] = merged_df['tamaño_pupila'].apply(identificar_valor)

    #Speed Interpolation
    Runningtimestamps_speed = pd.merge(runningtimestamps, runningspeed, left_index=True, right_index=True)
    f = interpolate.interp1d(Runningtimestamps_speed.index, Runningtimestamps_speed['speed'], kind='linear', fill_value="extrapolate")  # Ajusta 'tamaño_pupila' al nombre de la columna correspondiente en DF2
    merged_df['speed'] = f(merged_df['timestamps'])

    # Define function for categorising
    def category_pupil(value):
        if value >= 30:
            return 'Large pupil'
        elif value < 30:
            return 'Small pupil'
        elif value=='NaN':
            return 'NaN'

    # Apply the function to the column 'Value' to create the new column 'Category'.
    merged_df['pupil_size_category'] = merged_df['tamaño_pupila'].apply(category_pupil)

    # Define function for categorising
    def category_speed(value):
        if value > 0 :
            return 'Running'
        elif value <= 0:
            return 'No Running'
        elif value=='NaN':
            return 'NaN'

    # Apply the function to the column 'Value' to create the new column 'Category'.
    merged_df['speed_category'] = merged_df['speed'].apply(category_speed)
    
    # SAVE THE DATAFRAME IN A PARQUET FILE
    # path and name of the JSON file you want to save the DataFrame to
    ruta_archivo = "C:/Users/FelipeMartin/OneDrive - University of Sussex/DISSERTATION/segundo intento/12155577/boutons/"
    merged_df.to_parquet(ruta_archivo+folder+".parquet", engine='fastparquet')

['SS078-2017-10-04-001', 'SS075-2017-09-27-001', 'SS066-2016-09-01-001', 'SS069-2016-10-11-001', 'SS076-2017-10-04-001', 'SS070-2016-10-18-001', 'SS077-2017-10-03-001', 'SS069-2016-10-13-001', 'SS071-2016-10-18-001', 'SS075-2017-09-13-001', 'SS078-2017-09-28-001', 'SS077-2017-10-05-001', 'SS078-2017-10-05-001', 'SS076-2017-09-12-001', 'SS069-2016-10-21-001', 'SS076-2017-09-27-001', 'SS076-2017-10-02-001', 'SS077-2017-09-28-001']
C:/Users/FelipeMartin/OneDrive - University of Sussex/DISSERTATION/segundo intento/12155577/boutons/SS078/2017-10-04/001
       timestamps                                        folder_name  \
0           1.477  C:/Users/FelipeMartin/OneDrive - University of...   
1           1.613  C:/Users/FelipeMartin/OneDrive - University of...   
2           1.748  C:/Users/FelipeMartin/OneDrive - University of...   
3           1.883  C:/Users/FelipeMartin/OneDrive - University of...   
4           2.019  C:/Users/FelipeMartin/OneDrive - University of...   
...           

unique_count
40
C:/Users/FelipeMartin/OneDrive - University of Sussex/DISSERTATION/segundo intento/12155577/boutons/SS070/2016-10-18/001
       timestamps                                        folder_name  \
0           1.525  C:/Users/FelipeMartin/OneDrive - University of...   
1           1.662  C:/Users/FelipeMartin/OneDrive - University of...   
2           1.800  C:/Users/FelipeMartin/OneDrive - University of...   
3           1.937  C:/Users/FelipeMartin/OneDrive - University of...   
4           2.074  C:/Users/FelipeMartin/OneDrive - University of...   
...           ...                                                ...   
23563    5041.859  C:/Users/FelipeMartin/OneDrive - University of...   
23564    5041.996  C:/Users/FelipeMartin/OneDrive - University of...   
23565    5042.134  C:/Users/FelipeMartin/OneDrive - University of...   
23566    5042.271  C:/Users/FelipeMartin/OneDrive - University of...   
23567    5042.408  C:/Users/FelipeMartin/OneDrive - University of...   

unique_count
17
C:/Users/FelipeMartin/OneDrive - University of Sussex/DISSERTATION/segundo intento/12155577/boutons/SS078/2017-09-28/001
       timestamps                                        folder_name  \
0           1.466  C:/Users/FelipeMartin/OneDrive - University of...   
1           1.601  C:/Users/FelipeMartin/OneDrive - University of...   
2           1.736  C:/Users/FelipeMartin/OneDrive - University of...   
3           1.872  C:/Users/FelipeMartin/OneDrive - University of...   
4           2.007  C:/Users/FelipeMartin/OneDrive - University of...   
...           ...                                                ...   
28258    5028.034  C:/Users/FelipeMartin/OneDrive - University of...   
28259    5028.169  C:/Users/FelipeMartin/OneDrive - University of...   
28260    5028.304  C:/Users/FelipeMartin/OneDrive - University of...   
28261    5028.440  C:/Users/FelipeMartin/OneDrive - University of...   
28262    5028.575  C:/Users/FelipeMartin/OneDrive - University of...   

KeyError: '_ss_grating.intervals'